<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/TDGClusteringJerarquicoV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hdbscan

In [2]:
!pip install gower

In [3]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import hdbscan
import gower

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
!ls -l /content/gdrive/MyDrive/TDGdata/

total 11917268
-rw------- 1 root root    1108073 Apr 11 21:27 dfDims.csv
-rw------- 1 root root     795978 Apr 11 21:27 dfNoDims.csv
-rw------- 1 root root    2569549 Apr 11 21:27 k2Dims.csv
-rw------- 1 root root    1506267 Apr 11 21:27 k2NoDims.csv
-rw------- 1 root root    2569549 Apr 11 21:27 k3Dims.csv
-rw------- 1 root root    1506267 Apr 11 21:27 k3NoDims.csv
-rw------- 1 root root 5596536100 Apr 11 23:46 scores2.csv
-rw------- 1 root root 6596688400 Apr 11 23:40 scores.csv


In [6]:
# Matriz de distancias (disimilitud)
df = pd.read_csv("/content/gdrive/MyDrive/TDGdata/scores.csv", header = None)

In [7]:
df.iloc[0][0:5]

0    0.000000
1    0.016479
2    0.591425
3    0.000673
4    1.002986
Name: 0, dtype: float64

In [8]:
df.shape

(16244, 16244)

In [9]:
# Clustering jerarquico
clusterer = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=100)

In [10]:
clusterer.fit(df)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='precomputed', min_cluster_size=100, min_samples=None, p=None,
        prediction_data=False)

In [11]:
np.unique(clusterer.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [12]:
# Dataset de inventarios sin dimensiones de los productos
dataNoDims = pd.read_csv("/content/gdrive/MyDrive/TDGdata/dfNoDims.csv")

In [13]:
dataNoDims.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR,CONTRPERC
0,17,IG56,HTS119,1/1,B,PC,CAT17,-289.8802,-2.63,0.01
1,17,IG56,HTS119,1/1,B,PC,CAT17,289.5527,25.00,0.00
2,17,IG56,HTS119,1/1,C,PC,CAT17,1111.3121,50.00,0.00
3,17,IG56,HTS119,1/1,B,PC,CAT17,-266.8633,-9.09,0.00
4,15,IG56,HTS119,1/1,C,PC,CAT17,-517.0436,-11.11,0.00


In [14]:
dataNoDims.shape

(16244, 10)

In [15]:
# Get the position of categorical columns
catColumnsPos = [dataNoDims.columns.get_loc(col) for col in list(dataNoDims.select_dtypes('object').columns)]
print('Categorical columns : {}'.format(list(dataNoDims.select_dtypes('object').columns)))
print('Categorical columns position : {}'.format(catColumnsPos))

Categorical columns : ['IG', 'HTS', 'PLANNING', 'ABC', 'UOM', 'CAT']
Categorical columns position : [1, 2, 3, 4, 5, 6]


In [16]:
# Get the position of numerical columns
numColumnsPos = [dataNoDims.columns.get_loc(col) for col in list(dataNoDims.select_dtypes('float64').columns)]
print('Numerical columns : {}'.format(list(dataNoDims.select_dtypes('float64').columns)))
print('Numerical columns position : {}'.format(numColumnsPos))

Numerical columns : ['COSTX', 'QTYVAR', 'CONTRPERC']
Numerical columns position : [7, 8, 9]


In [17]:
numdf = dataNoDims[list(dataNoDims.select_dtypes('float64').columns)]

In [18]:
scaled_features = StandardScaler().fit_transform(numdf.values)
scaled_features_df = pd.DataFrame(scaled_features, index=numdf.index, columns=numdf.columns)
scaled_features_df

,COSTX,QTYVAR,CONTRPERC
0,-0.029671,-0.037167,-0.185651
1,0.092655,-0.006905,-0.210139
2,0.266138,0.020477,-0.210139
3,-0.024812,-0.044242,-0.210139
4,-0.077628,-0.046455,-0.210139
...,...,...,...
16239,0.098319,-0.033191,-0.210139
16240,0.163875,-0.033191,-0.210139
16241,1.078580,0.115063,-0.210139
16242,-0.247688,-0.143812,-0.210139


In [19]:
catdf = dataNoDims[list(dataNoDims.select_dtypes('object').columns)]

In [20]:
print("catdf",catdf.shape)
print("scaled_features_df",scaled_features_df.shape)
print(catdf.head(5))
print(scaled_features_df.head(5))
print(catdf.tail(5))
print(scaled_features_df.tail(5))
dataNoDimsEst = pd.concat((catdf, scaled_features_df), axis = 1)
dataNoDimsEst

catdf (16244, 6)
scaled_features_df (16244, 3)
     IG     HTS PLANNING ABC UOM    CAT
0  IG56  HTS119      1/1   B  PC  CAT17
1  IG56  HTS119      1/1   B  PC  CAT17
2  IG56  HTS119      1/1   C  PC  CAT17
3  IG56  HTS119      1/1   B  PC  CAT17
4  IG56  HTS119      1/1   C  PC  CAT17
      COSTX    QTYVAR  CONTRPERC
0 -0.029671 -0.037167  -0.185651
1  0.092655 -0.006905  -0.210139
2  0.266138  0.020477  -0.210139
3 -0.024812 -0.044242  -0.210139
4 -0.077628 -0.046455  -0.210139
        IG    HTS PLANNING ABC UOM   CAT
16239  IG5  HTS67      1/0   C  BX  CAT2
16240  IG5  HTS67      1/0   C  BX  CAT2
16241  IG5  HTS67      1/0   C  BX  CAT2
16242  IG5  HTS67      1/0   C  BX  CAT2
16243  IG5  HTS67      1/0   C  BX  CAT2
          COSTX    QTYVAR  CONTRPERC
16239  0.098319 -0.033191  -0.210139
16240  0.163875 -0.033191  -0.210139
16241  1.078580  0.115063  -0.210139
16242 -0.247688 -0.143812  -0.210139
16243 -0.022349 -0.089049  -0.210139


,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR,CONTRPERC
0,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167,-0.185651
1,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905,-0.210139
2,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477,-0.210139
3,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242,-0.210139
4,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455,-0.210139
...,...,...,...,...,...,...,...,...,...
16239,IG5,HTS67,1/0,C,BX,CAT2,0.098319,-0.033191,-0.210139
16240,IG5,HTS67,1/0,C,BX,CAT2,0.163875,-0.033191,-0.210139
16241,IG5,HTS67,1/0,C,BX,CAT2,1.078580,0.115063,-0.210139
16242,IG5,HTS67,1/0,C,BX,CAT2,-0.247688,-0.143812,-0.210139


In [21]:
# Matriz de distancias de Gower
gowerDistMatrix = gower.gower_matrix(dataNoDimsEst)

In [22]:
gowerDistMatrix = np.float64(gowerDistMatrix)

In [23]:
clusterer2 = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=100)

In [24]:
clusterer2.fit(gowerDistMatrix)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='precomputed', min_cluster_size=100, min_samples=None, p=None,
        prediction_data=False)

In [25]:
np.unique(clusterer2.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41])